# 7 - Matrix Rank
- 7.1 Six things about matrix rank
- 7.2 Interpretations of matrix rank
- 7.3 Computing matrix rank
- 7.4 Rank and scalar multiplication
- 7.5 Rank of added matrices
- 7.6 Rank of multiplied matrices
- 7.7 Rank of $A, A^T, A^T A, A A^T$
- 7.8 Rank of random matrices
- 7.9 Boosting rank by shifting
- 7.10 Rank difficulties
- 7.11 Rank and span
- 7.12 Exercises
- 7.13 Answers
- 7.14 Code challenges
- 7.15 Code solutions

Notes, code snippets, and the end of chapter exercises from the book _Linear Algebra: Theory, Intuition, Code_ by Mike X Cohen. 

Find more information about the book on [github](https://github.com/mikexcohen/LinAlgBook) and [amazon](https://www.amazon.com/Linear-Algebra-Theory-Intuition-Code/dp/9083136604).

In [1]:
import numpy as np

## 7.1 Six things about matrix rank

### 1/ Rank is a non-negative integer
* $0 \le \text{rank}(A)$
* Only the zeros matrix has a rank of 0.

### 2/ Maximum possible rank is min(M, N)
* $ 0 \le \text{rank}(A) \le \text{min}(M, N) $

### 3/ Rank is property of the entire matrix.

### 4/ Matrices are full rank when rank equals min(M, N)
* If $A \in R^{M,N}$, then the matrix is full rank when $\text{rank}(A) = \text{min}(M, N)$

### 5/ Rank indicates number of dimensions of information in the matrix.
* Rows or columns that are linearly dependent do not add to the rank of the matrix.

### 6/ Rank of a matrix is largest number of rows/cols that form a linearly independent set.

Notes
* One of the main goals of regularization is to add information to the matrix to increase stability or prevent overfitting.

## 7.2 Interpretations of matrix rank

### Geometric
Rank is the dimensionality of the subspace spanned by the columns (or rows) of the matrix.
* Compute the dimensionality by finding the number of vectors that form a linearly independent set.
* Count by row vectors or count by column vectors will always produce the same rank.

## 7.3 Computing matrix rank
Methods to compute the rank of a matrix.
1. Count the number of columns (or rows) that form a linearly independent set.
2. Count the number of pivots in the row-reduced echelon form of the matrix (result of Gaussian elimination).
3. Count the number of nonzero singular values from a singular value decomposition.

In [2]:
m, n = 4, 3
A = np.random.random((m,n))

# Compute the matrix rank.
r1 = np.linalg.matrix_rank(A)

# Factorize A into SVD.
U, Sigma, VT = np.linalg.svd(A)
r2 = np.count_nonzero(Sigma)

np.testing.assert_equal(r1, r2, "count nonzero singular values")

## 7.4 Rank and scalar multiplication
Scalar multiplication has no impact on the matrix rank except when the scalar is 0.

$$
\text{rank}(\alpha \textbf{A}) = \text{rank}(\textbf{A}), \quad \alpha \neq 0
$$

In [3]:
m, n = 4, 3
alpha = np.random.random(1)
A = np.random.random((m,n))

r1 = np.linalg.matrix_rank(A)
r2 = np.linalg.matrix_rank(alpha*A)

np.testing.assert_equal(r1, r2, "scalar multiplication does not change rank")

## 7.5 Rank of added matrices
$$
\text{rank}(\textbf{A} + \textbf{B}) \leq \text{rank}(\textbf{A}) + \text{rank}(\textbf{B})
$$

## 7.6 Rank of multiplied matrices
$$
\text{rank}(\textbf{A} \textbf{B}) \leq \text{min} ( \text{rank}(\textbf{A}), \text{rank}(\textbf{B}) )
$$

## 7.7 Rank of $A, A^T, A^T A, A A^T$
$$
\text{rank}(A) = \text{rank}(A^T) = \text{rank}(A^T A) = \text{rank}(A A^T)
$$

Notes
- Proof of $\text{rank}(A) = \text{rank}(A^T)$ is given by the fact that rank is property of matrix, not rows or columns.
- Proof of $\text{rank}(A) = \text{rank}(A^T A)$ is given by the SVD.
    - Number of nonzero singular values of $A$ is the same as the number of nonzero singular values of the covariance matrix of $A^T A$.

In [4]:
A = np.array([[1,3,4],[3,9,12]], dtype=int)

np.testing.assert_equal(np.linalg.matrix_rank(A), np.linalg.matrix_rank(A.T))
np.testing.assert_equal(np.linalg.matrix_rank(A), np.linalg.matrix_rank(A.T @ A))
np.testing.assert_equal(np.linalg.matrix_rank(A), np.linalg.matrix_rank(A @ A.T))

## 7.8 Rank of random matrices
Random matrices of floating point values are almost always full rank.
- Rationale: Rank reflects the amount of information in a matrix.

## 7.9 Boosting rank by shifting
Shifted matrix $\hat{A}$ is the result of adding a (small) multiple of the identity matrix to the matrix $A$.

$$
\hat{A} = A + \lambda I
$$

- Rationale: Adding a multiple of the full rank identity matrix increases the rank of the sum eg $\text{rank}(A + \lambda I) \ge \text{rank}(A)$.

In [5]:
A = np.array([[2,-4,10],[2,3,-4],[4,2,0]], dtype=float)
AplusI = A + 0.01 * np.eye(3)

print(f"rank(A)     : {np.linalg.matrix_rank(A)}")
print(f"rank(AplusI): {np.linalg.matrix_rank(AplusI)}")

rank(A)     : 2
rank(AplusI): 3


## 7.11 Rank and span
Method to determine whether a vector $v$ is in the span of the vectors formed by the column space or row space of the matrix $A$.
- Compute the $\text{rank}(\textbf{A})$.
- Compute the rank of the augmented matrix $\text{rank}(\textbf{A}|\textbf{v})$.
- If the rank of the augmented matrix is equal to the original matrix, then $\textbf{v}$ is in the span of the vectors of $\textbf{A}$, else $\textbf{v}$ is not in the span of $A$.

## 7.14 Code challenges

> 1. The goal of this code challenge is to create random matrices with any arbitrary rank. In particular, combine matrix multiplication with the rule about rank and matrix multiplication eg $\text{rank}(\textbf{A} \textbf{B}) \leq \text{min} ( \text{rank}(\textbf{A}), \text{rank}(\textbf{B}) )$ to create reduced-rank matrices comprising random numbers.

In [6]:
def rank(m, n, r):
    """
    rank returns a matrix of random values with dimensions m \times n and rank r 

    :param m: int            Number of rows.
    :param n: int            Number of columns.
    :param r: int            Rank.
    :return: numpy.ndarray   Matrix with dimensions m \times n and rank r.
    """
    assert r <= m and r <=n

    A = np.random.random((m, r))
    B = np.random.random((r, n))
    return A @ B


# Generate 4 x 4 matrices with increasing rank from 1..4. 
for r in [1, 2, 3, 4]:
    Ar = rank(4, 4, r)
    np.testing.assert_equal(np.linalg.matrix_rank(Ar), r, err_msg="rank(A) != r")

> 2. The goal of this code challenge is to explore the tolerance level of your computer for computing the rank of matrices with tiny values. Start by creating the $5 \times 5$ zeros matrix and confirm that the rank is 0. Then add a $5 \times 5$ matrix of random values scaled by machine epsilon and confirm the the rank is 5. Keep scaling the machine epsilon down until the rank of the summed matrix is 0. Compute the Frobenius norm of the summed matrix to get a sense of the magnitude of the values in the matrix.

In [7]:
# Create a zeros matrix and confirm the rank is 0.
n = 5
A = np.zeros((n,n))
np.testing.assert_equal(np.linalg.matrix_rank(A), 0, err_msg="rank(A) != 0")

# Add a full rank matrix scaled by machine epsilon and confirm the rank is n.
eps, denom = np.finfo(float).eps, 1.
B = eps/denom * np.random.random((n,n))
AplusB = A + B
r = np.linalg.matrix_rank(AplusB)
AplusBnorm = np.linalg.norm(AplusB, 'fro')
np.testing.assert_equal(r, n, err_msg="rank(A+B) != n")
print(f"rank(A+B): {r} ||A+B||: {AplusBnorm} denom: {denom}")

# Shrink machine epsilon until the rank of the sum is 0.
while r > 0:
    denom *= 2.
    B = eps/denom * np.random.random((n,n))
    AplusB = A + B
    r = np.linalg.matrix_rank(AplusB)
    AplusBnorm = np.linalg.norm(AplusB, 'fro')
    
print(f"rank(A+B): {r} ||A+B||: {AplusBnorm} denom: {denom}")

rank(A+B): 5 ||A+B||: 6.182731519641867e-16 denom: 1.0
rank(A+B): 0 ||A+B||: 0.0 denom: 8.98846567431158e+307
